### Bayesian optimization 

In [ ]:
import scipy.optimize
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.preprocessing import OneHotEncoder
import sklearn
import GPy
import matplotlib.pyplot as plt

In [ ]:
class Toy_function:
    '''
    This function is called Ackley, it's a standard benchmark function for optimizers.
    '''
    def __init__(self, ndim):
        self.ndim = ndim
        self.a = 20
        self.b = 0.2
        self.c = 2 * np.pi
        self.domain = [[-32.768, 32.768] for ii in range(self.ndim)]
        
    def f(self, X):
        if not isinstance(X, np.ndarray):
            raise TypeError("X have to be a 1D numpy array !")
        if X.ndim == 2 and X.shape[0] != 1:
            raise ValueError("Pass input values one by one")
        if X.ndim == 2:
            X = X.reshape(-1)
            
        if X.shape[0] != self.ndim:
            raise ValueError
        res = - self.a * np.exp(-self.b * np.sqrt((1/self.ndim) * np.sum(X**2))) - np.exp((1/self.ndim) * np.sum(np.cos(self.c * X))) + self.a + np.exp(1)
        return np.array(res).reshape(-1, 1)

# how to use it ? 
function_to_optimize = Toy_function(ndim=2)
function_to_optimize.f(np.array([[0, 0]]).reshape(-1, 2))

### Random optimization

In [ ]:
## Code Here
## Input : 
#####
## f: callable 
## domain: input space (bounds)
## n: number of drawn element
#####
## Output : 
## X : np.ndarray (n, dim_domain)
## Y : np.ndarray (n,)
#####
def draw_random(f, domain, n):
    # TODO 
    return X, Y

# How to use it
nb_init = 5
X_init, Y_init = draw_random(f, domain, nb_init)
print(X_init)
print(Y_init)

### Create your surrogate model

In [ ]:
## Code here
## See GPy GPRegression documentation
## Hint :
## - You need X, Y, kernel 
## - GP supports unnormalized data but it can mislead the fitting process (StandardScaler is a good option)
## - A standard kernel for a smooth regression is Matern52, see GPy.kern.Matern52
## - You need to optimize hyper parameters and following the log marginal likelihood, see model.optimize_restarts
class My_model:
    def __init__(self, input_dim, num_restarts_optim_model=10):
        # TODO
    
    def fit(self, X, Y):
        # TODO
    
    def predict(self, X):
        # TODO
        return mu, sig
    
    def get_model(self):
        return self.model
    
    def get_kernel(self):
        return self.kernel # or self.model.kernel

In [ ]:
#mymodel = My_model(input_dim=len(domain))
#mymodel.fit(X_init, Y_init)
#mymodel.get_model()

### Create your acqusition function

In [ ]:
## Code here
## Minimization
class LCB_acquisition:
    def __init__(self, model, exploration_param=0.05):
        # TODO
        
        
    def acq(self, X):
        # TODO
        return acq_value

### Optimize your acquisition function

In [ ]:
## Code here
def optimize_acquisition(acq, domain):
    
    # Setup f_to_optimize with a wrapper for convenience
    def f_wrapper(acq_obj):
        def f(X):
            return - acq_obj.acq(X).reshape(-1) # maximize <==> minimize the negative
        return f
    
    f_to_optim = f_wrapper(acq)
    
    # TODO
    
    return x_opt, f_opt
        
#acquisition = LCB_acquisition(mymodel)
#res = optimize_acquisition(acquisition, domain)

### All together : Bayesian optimization algorithm

In [ ]:
## Code here
def bayesian_optimization(func, domain, budget=50):
    # TODO
    return X, Y


function = Toy_function(ndim=2)
f = function.f
domain = function.domain

X, Y = bayesian_optimization(f, domain)

#### Analysis 

In [ ]:
def plot_best_score_evolution(Y):
    best_y = []
    for yy in Y:
        if len(best_y) == 0:
            best_y.append(yy)
            continue
        if yy < best_y[len(best_y) - 1]:
            best_y.append(yy)
        else:
            best_y.append(best_y[len(best_y) - 1])
    
    plt.plot(best_y)
    plt.show()
plot_best_score_evolution(Y)

In [ ]:
class Model_to_optimize():
    
    def __init__(self):
        self.list_names_param = [
            "min_samples_split",
            "min_samples_leaf",
            "min_weight_fraction_leaf",
            "max_features",
            "max_samples"]
        self.domain = [
            (10e-5, 1 - 10e-5), 
            (10e-5, 0.5),
            (10e-5, 0.5),
            (10e-5, 1 - 10e-5), 
            (10e-3, 1 - 10e-5) #--
        ]
        df = pd.read_csv("./laptop_price_clean.csv")
        X = df.drop(columns=['Price_euros'])
        self.X_data = pd.get_dummies(X)
        self.Y_data = np.log(df['Price_euros'])

    '''
        Parameters_array : numpy array containing 0+ values correspind to the parameters of the RF (if Ø or None, default values are used)
        Return: sklearn readable parameters
    '''
    def preprocess_input_parameters(self, parameters_array):
        self.params = {}
        for ii in range(parameters_array.shape[0]):
            if parameters_array[ii] is not None and (parameters_array[ii] < self.domain[ii][0] or parameters_array[ii] > self.domain[ii][1]):
                raise ValueError("parameter not in bound")
            if parameters_array[ii] is not None:
                self.params[self.list_names_param[ii]] = parameters_array[ii]

    def pipeline(self, X_data):
        pass
    
    def f(self, X):
        if not isinstance(X, np.ndarray):
            raise TypeError("X have to be a 1D numpy array !")
        if X.ndim == 2 and X.shape[0] != 1:
            raise ValueError("Pass input values one by one")
        if X.ndim == 2:
            X = X.reshape(-1)
            
        self.preprocess_input_parameters(X)
        kf = sklearn.model_selection.KFold(n_splits=5)
        res = []
        for train_index, test_index in kf.split(self.X_data):
            
            X_train = self.X_data.iloc[train_index]
            X_test = self.X_data.iloc[test_index]
            
            Y_train = self.Y_data.iloc[train_index]
            Y_test = self.Y_data.iloc[test_index]
            
            model = ensemble.RandomForestRegressor(**self.params)
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)
            try:
                metric = sklearn.metrics.r2_score(Y_test, Y_pred)
                res.append(metric)
            except ValueError:
                print(self.params)

        score = np.mean(res) #- np.std(res) 
        # Maximize (mean(R2 score) - std(R2 score)) <==> Minimize it's negative
        return np.array( - score).reshape(-1, 1)


In [ ]:
model_to_optimize = Model_to_optimize()
domain = model_to_optimize.domain

X, Y = bayesian_optimization(model_to_optimize.f, domain)

In [ ]:
plot_best_score_evolution(Y)